# Import bibliothèques

In [1]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

# Chemins des fichiers

In [2]:
dataset='C:\Dossier Enora\Travail\VS Code\Projet LSF\model\dataset.csv'
model_save_path='C:\Dossier Enora\Travail\VS Code\Projet LSF\model\classification.hdf5'
tflite_save_path='C:\Dossier Enora\Travail\VS Code\Projet LSF\model\classification.tflite'

# Nombre de classe : nombre de signe que l'on peut reconnaitre

In [3]:
NUM_CLASSES=21      #nombre de signes différents

# Lecture du dataset

In [4]:
X_data=np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21*2)+1)))       #les coordonnées normmalisées des landmarks
y_data=np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))       #les id des labels 


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.6, random_state=RANDOM_SEED)        #choisir une random seed permet de s'assurer de la répétabilité du split des donénes pour pouvoir évaluter deux modèles différents sur la meme base
y_test

array([14,  7,  5, ..., 11,  1, 11])

# Création du modèle

In [68]:
#version 1.0

# model = tf.keras.models.Sequential([
#     tf.keras.layers.Input((21*2, )),        #spécifier la taille de l'entrée
#     tf.keras.layers.Dropout(0.2),   #permet d'éviter le surapprentissage (20% des neuronnes de la couche précédente seront désactivés pendant l'entrainement)
#     tf.keras.layers.Dense(20, activation='relu'),       #couche dense de 20 neurones avec une fonction d'activation ReLU (fonction non linéaire qui introduire de la non-linéarité dans le modèle)
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(10, activation='relu'),       #couche dense de 10 neurones avec une fonction d'activation ReLU
#     tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')     #couche de sortie avec une fonction d'activation softmax (permet de normaliser les valeurs de sortie entre 0 et 1)
# ])

In [7]:
#version 2.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21*2, )),  # Spécifier la taille de l'entrée
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                1376      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 flatten (Flatten)           (None, 128)               0

In [9]:
#call back du modèle
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path, save_weights_only=False, verbose=1)      #le modèle complet est sauvegardé
                                                                                                                    #verbose=1 => messsage pour indiquer quand le modèle à été sauvegardé
#arrêter l'entrainement prématurément si certaines conditions ne sont pas satisfaites                                                              
es_callback = tf.keras.callbacks.EarlyStopping(patience=40, verbose=1)      #patience=20 => arrêter l'entrainement si la précision n'a pas augmenté pendant 20 epochs

In [10]:
#compilation du modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])       #optimizer='adam' => algorithme d'optimisation de gradient stochastique
                                                                                                    #loss='sparse_categorical_crossentropy' => fonction de perte pour les problèmes de classification
                                                                                                    #metrics=['accuracy'] => métrique pour évaluer le modèle

# Entrainement du modèle

In [11]:
model.fit(
    X_train, 
    y_train, 
    epochs=1000,                                #epochs => nombre de fois que le modèle va voir l'ensemble des données, nombre d'itérations
    validation_data=(X_test, y_test), 
    callbacks=[cp_callback, es_callback]
)       

Epoch 1/1000
26/26 [==============================] - ETA: 0s - loss: 3.1197 - accuracy: 0.0881     
Epoch 1: saving model to C:\Dossier Enora\Travail\VS Code\Projet LSF\model\classification.hdf5
26/26 [==============================] - 2s 17ms/step - loss: 3.1197 - accuracy: 0.0881 - val_loss: 3.0258 - val_accuracy: 0.1934
Epoch 2/1000
 1/26 [>.............................] - ETA: 0s - loss: 3.0373 - accuracy: 0.3125
Epoch 2: saving model to C:\Dossier Enora\Travail\VS Code\Projet LSF\model\classification.hdf5
26/26 [==============================] - 0s 6ms/step - loss: 2.9500 - accuracy: 0.1625 - val_loss: 2.8581 - val_accuracy: 0.1760
Epoch 3/1000
 1/26 [>.............................] - ETA: 0s - loss: 2.9515 - accuracy: 0.1875
Epoch 3: saving model to C:\Dossier Enora\Travail\VS Code\Projet LSF\model\classification.hdf5
26/26 [==============================] - 0s 5ms/step - loss: 2.7837 - accuracy: 0.2097 - val_loss: 2.6763 - val_accuracy: 0.2083
Epoch 4/1000
 1/26 [>.............

# Evaluation du modèle

In [12]:
val_loss,val_acc=model.evaluate(X_test, y_test, batch_size=128)       #batch_size => nombre d'échantillons qui vont être propagés dans le réseau

10/10 [==============================] - 0s 2ms/step - loss: 0.2125 - accuracy: 0.9529


In [13]:
#test d'inférence (prédiction) sur un exemple => ici, cela devrait prédire 0, on prend en compte la première ligne du dataset
prediction=model.predict(np.array([X_test[0]]))
print(np.squeeze(prediction))       #np.squeeze => supprime les dimensions unitaires d'un tableau => affichage des probabilités de chaque classe
print(f"l'identifiant de la classe (signe) prédite est : {np.argmax(prediction)}")        #np.argmax => renvoie l'indice de la valeur maximale d'un tableau => affichage de la classe prédite

1/1 [==============================] - 0s 166ms/step
[5.7639986e-25 8.1269304e-17 2.5174873e-12 1.3750664e-27 1.3044698e-30
 3.8274588e-06 0.0000000e+00 0.0000000e+00 1.2540913e-20 0.0000000e+00
 1.6160322e-28 2.7426783e-13 3.1602828e-11 1.5855839e-15 9.9999619e-01
 1.0803584e-31 2.7582389e-35 9.1802712e-14 0.0000000e+00 1.5683672e-15
 2.1082147e-16]
l'identifiant de la classe (signe) prédite est : 14


# Conversion du modèle en un modèle Tensorflow-Lite

In [14]:
#enregistrement du modèle
model.save(model_save_path, include_optimizer=False)

In [15]:
#transformation du modèle en tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]        #optimisation du modèle pour améliorer ses performances sur les dispositifs mobiles et les systèmes embarqués
tflite_quantized_model = converter.convert()

open(tflite_save_path, "wb").write(tflite_quantized_model)

INFO:tensorflow:Assets written to: C:\Users\enora\AppData\Local\Temp\tmprc_6oz64\assets


INFO:tensorflow:Assets written to: C:\Users\enora\AppData\Local\Temp\tmprc_6oz64\assets


18208

# Test de prédiction (Inference)

In [16]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [17]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [18]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [19]:
#implementation de la prédiction
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

In [20]:
print(np.squeeze(tflite_results))
print(f"l'identifiant de la classe (signe) prédite est : {np.argmax(np.squeeze(tflite_results))}")
print(f"indice de confiance de la prédiction : {np.max(np.squeeze(tflite_results))*100} %")

[6.83481464e-25 1.09764300e-16 2.78564606e-12 2.46883997e-27
 2.01635085e-30 4.57123315e-06 0.00000000e+00 0.00000000e+00
 1.83003048e-20 0.00000000e+00 2.75968909e-28 3.24494880e-13
 4.78786767e-11 1.87518462e-15 9.99995470e-01 2.31559551e-31
 4.56515399e-35 1.06745714e-13 0.00000000e+00 1.80133986e-15
 2.94655816e-16]
l'identifiant de la classe (signe) prédite est : 14
indice de confiance de la prédiction : 99.9995470046997 %
